# OPENCV,mediapipe를 활용한 손가락 관절 인식

In [5]:
import cv2
import mediapipe as mp

cap=cv2.VideoCapture(0)
mpHands=mp.solutions.hands

hands=mpHands.Hands()
print(hands)

mpDraw=mp.solutions.drawing_utils

while True:
    success,img=cap.read()
    #openCV는 RGB를 BGR로 저장함
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results=hands.process(imgRGB)
    print(results.multi_hand_landmarks)
    #손가락을 인식했다면
    if results.multi_hand_landmarks:
        for i in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img,i)
    cv2.imshow('text',img)
    #1밀리초마다 0xFF비트를 검사해서 q 눌렀으면 종료
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
[landmark {
  x: 0.735621691
  y: 1.05562091
  z: 7.86787552e-007
}
landmark {
  x: 0.667090595
  y: 1.00759423
  z: -0.036245957
}
landmark {
  x: 0.610035062
  y: 0.919132829
  z: -0.067906484
}
landmark {
  x: 0.580716133
  y: 0.822682
  z: -0.0919275954
}
landmark {
  x: 0.565053582
  y: 0.740825
  z: -0.117579818
}
landmark {
  x: 0.668376
  y: 0.858430743
  z: -0.109669313
}
landmark {
  x: 0.641653299
  y: 0.715737104
  z: -0.154483125
}
landmark {
  x: 0.622894526
  y: 0.63028121
  z: -0.178213745
}
landmark {
  x: 0.60416615
  y: 0.56145221
  z: -0.194213822
}
landmark {
  x: 0.722158611
  y: 0.856607497
  z: -0.116017304
}
landmark {
  x: 0.687817931
  y: 0.718326867
  z: -0.166576415
}
landmark {
  x: 0.658521354
  y: 0.626095057
  z: -0.196207583
}
landmark {
  x: 0.632892728
  y: 0

# 랜드마크 간 선 그리기

In [6]:
import cv2
import mediapipe as mp

cap=cv2.VideoCapture(0)
mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw=mp.solutions.drawing_utils

while True:
    success,img=cap.read()
    #이미지 반전하는 함수임, 1=좌우반전, -1=상하반전
    img=cv2.flip(img,1)
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results=hands.process(imgRGB)
    
    if results.multi_hand_landmarks:
        for i in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img,i,mpHands.HAND_CONNECTIONS)
    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# 8번 관절 타게팅

In [7]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
cap=cv2.VideoCapture(0)
my_hands=mp.solutions.hands.Hands()
mpDraw=mp.solutions.drawing_utils

while True:
    success,img=cap.read()
    image=cv2.resize(img,(1920,1080))
    image=cv2.flip(image,1)
    f_height,f_width,idx=image.shape
    rgb_img=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    #손인식 결과
    output=my_hands.process(rgb_img)
    hands=output.multi_hand_landmarks
    if hands:
        for hand in hands:
            mpDraw.draw_landmarks(image,hand)
            landmarks=hand.landmark
            for id, landmark in enumerate(landmarks):
                x=int(landmark.x*f_width)
                y=int(landmark.y*f_height)
                if id==8:
                    cv2.circle(img=image,center=(x,y),radius=20,color=(0,255,0),thickness=10)

    cv2.imshow('img',image)
    # plt.imshow(image)
    # plt.axis('off')
    # plt.show()
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 클래스 직접구현(잡다한 기능을 붙힐 수 있음. 가장 최신의 손은 초록 점으로)

In [8]:
import cv2
import mediapipe as mp
cap=cv2.VideoCapture(0)
#클래스 직접 만들어서 써보기
class handDetector():
    def __init__(self,mode=False,maxHands=2,modelComplexity=1,detectionCon=0.5,trackCon=0.5):
        self.mode=mode
        self.maxHands=maxHands
        self.modelComplexity=modelComplexity
        self.detectionCon=detectionCon
        self.tranCon=trackCon
        self.mpHands=mp.solutions.hands
        self.hands=self.mpHands.Hands(self.mode,self.maxHands,self.modelComplexity,self.detectionCon,self.tranCon)
        self.mpDraw=mp.solutions.drawing_utils

    def findHands(self,img,draw=True):
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results=self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img,handLms,self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self,img,handN=0,draw=True):
        lmList=[]
        if self.results.multi_hand_landmarks:
            mpHands=self.results.multi_hand_landmarks[handN]
            for id, lm in enumerate(mpHands.landmark):
                h,w,c=img.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy])#21개 점 위치 저장하기
                if draw:
                    cv2.circle(img,(cx,cy),10,(155,255,0),cv2.FILLED)
        return lmList
detector=handDetector()

while True:
    flag,img=cap.read()
    img=cv2.flip(img,1)

    img=detector.findHands(img)
    lmList=detector.findPosition(img)
    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# 손가락 접힘 감지(그 전 관절 좌표와 비교하여)

In [9]:
import cv2
import mediapipe as mp

class handDetector():
    def __init__(self,mode=False,maxHands=2,modelComplexity=1,detectionCon=0.5,trackCon=0.5):
        self.mode=mode
        self.maxHands=maxHands
        self.modelComplexity=modelComplexity
        self.detectionCon=detectionCon
        self.trackCon=trackCon

        self.mpHands=mp.solutions.hands
        self.hands=self.mpHands.Hands(self.mode,self.maxHands,self.modelComplexity,self.detectionCon,self.trackCon)
        self.mpDraw=mp.solutions.drawing_utils

    def findhands(self,img,draw=True):
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results=self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img,handLms,self.mpHands.HAND_CONNECTIONS)
        return img
    def findPosition(self,img,handNo=0,draw=True):
        lmList=[]
        if self.results.multi_hand_landmarks:
            mpHand=self.results.multi_hand_landmarks[handNo]
            for id,lm in enumerate(mpHand.landmark):
                h,w,c=img.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy])
                if draw:
                    cv2.circle(img,(cx,cy),50,(155,255,155,cv2.FILLED))
        return lmList
cap=cv2.VideoCapture(0)
detector=handDetector()#따로 클래스를 구현해서 쓰도록 하자

tipid=[4,8,12,16,20]#손끝 id 번호


while True:
    flag,img=cap.read()
    img=cv2.flip(img,1)
    img=detector.findhands(img)
    lmList=detector.findPosition(img)

    if lmList:
        fingers=[]
        #엄지는 워낙 짧다 보니 바로 밑에꺼 하나에만 영향받게, x축으로 특별히
        if lmList[tipid[0]][1]>lmList[tipid[0]-1][1]:
            fingers.append(0)
        else:
            fingers.append(1)
        #엄지 외
        for id in range(1,5):
            if lmList[tipid[id]][2]<lmList[tipid[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        #print(fingers)
        cnt=fingers.count(1)
        # print(cnt)
        cv2.rectangle(img,(50,50),(200,200),(255,255,255),cv2.FILLED)
        cv2.putText(img,str(cnt),(75,180),cv2.FONT_HERSHEY_PLAIN,10,(0,0,0),20)
    

    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# 얼굴인식을 해보자

In [10]:
import cv2
import mediapipe as mp
cap=cv2.VideoCapture(0)
mpFaceDetection=mp.solutions.face_detection 
faceDetection=mpFaceDetection.FaceDetection(0.75) #1에 가까울 수록 인식을 엄격하게 함

mpDraw=mp.solutions.drawing_utils
while True:
    flag,img=cap.read()
    img=cv2.flip(img,1)
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results=faceDetection.process(imgRGB)
    print(results.detections)

    if results.detections:
        for id,detection in enumerate(results.detections):
            mybox=detection.location_data.relative_bounding_box
            myheight,mywidth,myc=img.shape
            cbox=int(mybox.xmin*mywidth),int(mybox.ymin*myheight),int(mybox.width*mywidth),int(mybox.height*myheight)
            cv2.rectangle(img,cbox,(0,255,0),2)
            cv2.putText(img,str(int(detection.score[0]*100))+'%',(cbox[0],cbox[1]-20),cv2.FONT_HERSHEY_PLAIN,2,(0,255,255),3)
            cv2.putText(img,'number:'+str(len(results.detections)),(mywidth-250,50),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),3)
    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



[label_id: 0
score: 0.908814728
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.38179791
    ymin: 0.503068447
    width: 0.251962841
    height: 0.335950434
  }
  relative_keypoints {
    x: 0.434457541
    y: 0.589194357
  }
  relative_keypoints {
    x: 0.536045492
    y: 0.602127135
  }
  relative_keypoints {
    x: 0.460647821
    y: 0.677191436
  }
  relative_keypoints {
    x: 0.463817179
    y: 0.746702671
  }
  relative_keypoints {
    x: 0.404259413
    y: 0.608852804
  }
  relative_keypoints {
    x: 0.626147628
    y: 0.639670372
  }
}
]
[label_id: 0
score: 0.913288295
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.381433308
    ymin: 0.505358517
    width: 0.247768342
    height: 0.330358267
  }
  relative_keypoints {
    x: 0.431069672
    y: 0.589761734
  }
  relative_keypoints {
    x: 0.531699359
    y: 0.599534214
  }
  relative_keypoints {
    x: 0.457043171
    y: 0.672807753
  }
  relative